In [1]:
from nltk.corpus import wordnet as wn
from lxml import etree
import numpy as np
import pandas as pd
from os import path
from collections import Counter

In [5]:
taxonomy = pd.read_csv('taxonomy.csv', header=0)
taxonomy.columns

Index(['liquid-alcohol-sweet', 'liquid-alcohol', 'liquid-juices',
       'liquid-juices-citric', 'liquid-syrups', 'solid-fruits',
       'solid-fruits-citric', 'solid-veggies', 'te-sweet', 'te', 'others'],
      dtype='object')

In [23]:
ingredients_list = {
    ('liquid', 'alcohol', 'sweet'): [],
    ('liquid', 'alcohol'): [],
    ('liquid', 'juices'): [],
    ('liquid', 'juices', 'citric'): [],
    ('liquid', 'syrups'): [],
    ('solid', 'fruits'): [],
    ('solid', 'fruits', 'citric'): [],
    ('solid', 'veggies'): [],
    ('te', 'sweet'): [],
    ('te',): [],
    ('others',): []}

In [25]:
tree = etree.parse('ccc_cocktails.xml')

recipes = tree.findall('recipe')
# process each sentence in the file
titles = []
all_ingredients = []
instances = pd.DataFrame()
cocktails = {}
for recipe in recipes:
    title = recipe[0]
    titles.append(title)
    ingredients = recipe[1]
    ingredients_list = {
        ('liquid', 'alcohol', 'sweet'): [],
        ('liquid', 'alcohol'): [],
        ('liquid', 'juices'): [],
        ('liquid', 'juices', 'citric'): [],
        ('liquid', 'syrups'): [],
        ('solid', 'fruits'): [],
        ('solid', 'fruits', 'citric'): [],
        ('solid', 'veggies'): [],
        ('te', 'sweet'): [],
        ('te',): [],
        ('others',): []
    }
    for ingredient in ingredients:
        ingredient_val = ingredient.values()[-1].lower()
        all_ingredients.append(ingredient_val)
        for col in taxonomy.columns:
            col_tup = tuple(col.split('-'))

            if taxonomy[col].isin([ingredient_val]).any():
                ingredients_list[col_tup].append(ingredient_val)
                break
    
    cocktails[title] = ingredients_list

In [35]:
recipe[0]

<Element title at 0x7df29fc5c8>

In [ ]:
taxonomy['Fruits-citric'].isin(['grapefruit']).any()

In [ ]:
len(titles), len(all_ingredients)

In [ ]:
unique_ingredients = set(all_ingredients)

In [ ]:
len(unique_ingredients), unique_ingredients

In [ ]:
words_found = []
words_notfound = []
for w in unique_ingredients:
    syn = wn.synsets(w)
    if not syn:
        words_notfound.append(w)
    else:
        words_found.append(w)


In [ ]:
len(words_notfound), len(words_found)

In [ ]:
ingredient_words = []
for ingr in all_ingredients:
    if '_' in ingr:
        print(ingr)
        ingr = ingr.replace('_',' ')
    if ingr == 'get_27':
        ingr = 'get27'
    elif ingr == 'manguo juice':
        ingr = 'mango juice'
    #elif ingr == 'cachaça':
    #    ingr = 'cachaca'
    ingredient_words.extend(ingr.split())

In [ ]:
def w2v_get_weights(words, n_words, emb_dim, w2v_model):
    c = 0
    not_found = []
    embedding_matrix = np.zeros((n_words, emb_dim))
    for i, word in enumerate(words):
        try:
            embedding_vector = w2v_model.wv[word]
        except KeyError:
            embedding_vector = None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            c += 1
            not_found.append(word)
    print(c, 'were not found')
    return embedding_matrix, not_found

In [ ]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

dir_folder = 'glove'
glove_name = 'glove_42B_300d.txt'
weights_file = path.join(dir_folder, glove_name.split('.')[0] + '.npy')
glove_file = path.join(dir_folder, glove_name)
emb_file = '2' + glove_name
w2v_glove_file = path.join(dir_folder, emb_file)


if not path.exists(w2v_glove_file):
    glove2word2vec(glove_file, w2v_glove_file)

if not path.exists(weights_file):
    w2v_model = KeyedVectors.load_word2vec_format(w2v_glove_file, binary=False)
    emb_weights, not_found = w2v_get_weights(ingredient_words,
                                len(ingredient_words),
                                300,
                                w2v_model)
    del w2v_model
    np.save(path.join(dir_folder,weights_file), emb_weights)
else:
    emb_weights = np.load(weights_file)

In [ ]:
embeddings_index = {}
with open(path.join(dir_folder, glove_name)) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [ ]:
not_found

In [ ]:
emb_weights.shape

In [ ]:
list_alchohol=['white rum','kirsch','Cremant','cognac','champagne','vodka','martini','Noilly_Prat','Whiskey','Benedictine','pastis','rum','malibu rum','dry white wine','Porto','dark rum','White martini','Angostura bitter','pisang ambon','cava','Prosecco','Amber rum','lemon liqueur','sparkling wine','campari','vermouth','Creme de café','triple sec','white wine','red martini','plum brandy','rice wine','calvados','cachaca']

list_fruits=['lemon','raspberry','citrus fruit','strawberry','kiwi fruit','orange','apricot','currant/black currant','lime','"lime zest(outer skin of lime)"','blood orange','pineapple apple','grapefruit','banana','litchi','berry','melon']

list_veggies=['cucumbers','lime','wasabi','lemongrass','ginger','mint','tomato','coriander','guava']

list_liqueur=['Lemonade','Blue curacao','syrup','passion fruit syrup','Orange juice','cointreau','grenadine','pineapple juice','coconut','lime juice','Apricot juice','Creme de cassis','apple juice','grapefruit juice','Hard cider','Cranberry juice','Banana juice','Mango juice','Passion fruit juice','rice milk','coconut milk','Tamarin juice','coffee','apple cider','apricot liqueur','worcestershire sauce','"Currant syrup(blackcurrant juice)"','orgeat syrup','grand marnier','litchi juice','berry juice','strawberry juice','coffee liqueur','coca-cola']

list_te=['sugar','cane sugar','sour cream','tabasco sauce','light whipping cream','egg','cinnamon','nutmeg','brown sugar','anise basil','powdered sugar','granulated sugar','vanilla sugar','celery salt','pepper','milk','salt']

list_others=['ice cube','sparkling water','ice cream','soda water','sparkling mineral water']


In [ ]:
len(list_alchohol), len(list_fruits), len(list_veggies), len(list_liqueur), len(list_te), len(list_others)

In [ ]:
count_ingredients = Counter(all_ingredients)
sorted(count_ingredients.items(), key=lambda pair: pair[1], reverse=True)

In [ ]:
all_categories = []
all_categories.extend(list_alchohol)
all_categories.extend(list_liqueur)
all_categories.extend(list_fruits)
all_categories.extend(list_veggies)
all_categories.extend(list_te)
all_categories.extend(list_others)

In [ ]:
len(all_categories), len(list_alchohol) + len(list_liqueur) + len(list_fruits) + len(list_veggies) + len(list_te) + len(list_others)

In [ ]:
all_categories = list(map(lambda x:x.lower(), all_categories))
for ingr in unique_ingredients:
    if ingr not in all_categories:
        print(ingr)

In [ ]:
import pandas as pd
categs = pd.concat([pd.Series(list_alchohol), pd.Series(list_liqueur), pd.Series(list_fruits), pd.Series(list_veggies), 
           pd.Series(list_te), pd.Series(list_others)], axis=1)
categs.columns = ['Alcohol', 'Liqueur', 'Fruits', 'Veggies', 'Taste Enhancers', 'Others']
categs.to_excel('Categs.xlsx')